<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

In [2]:
import os
os.chdir('C:\\Users\\prudi\\Desktop\\Data Sets\\quora-question-pairs')
df = pd.read_csv("train.csv")
df.shape

(404290, 6)

In [3]:
# avoid decoding problems
df = pd.read_csv("train.csv")
df=df.loc[0:100,]
 
# encode questions to unicode
# https://stackoverflow.com/a/6812069
# ----------------- python 2 ---------------------
# df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
# df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))
# ----------------- python 3 ---------------------
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [4]:
df.head()

id  qid1  qid2                                          question1  \
0   0     1     2  What is the step by step guide to invest in sh...   
1   1     3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   
2   2     5     6  How can I increase the speed of my internet co...   
3   3     7     8  Why am I mentally very lonely? How can I solve...   
4   4     9    10  Which one dissolve in water quikly sugar, salt...   

                                           question2  is_duplicate  
0  What is the step by step guide to invest in sh...             0  
1  What would happen if the Indian government sto...             0  
2  How can Internet speed be increased by hacking...             0  
3  Find the remainder when [math]23^{24}[/math] i...             0  
4            Which fish would survive in salt water?             0

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [15]:
len(word2tfidf)

787

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [30]:
i=0
for question1 in df['question1'][:1]:
    for word1 in question1:
        i+=1
print(i)

66


In [17]:
import warnings
# en_vectors_web_lg, which includes over 1 million unique vectors.
#nlp = spacy.load('en_core_web_sm')
import en_core_web_sm
nlp = en_core_web_sm.load()
vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), 384])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df['q1_feats_m'] = list(vecs1)


100%|████████████████████████████████████████████████████████████████████████████████| 101/101 [00:01<00:00, 61.56it/s]


In [26]:
df.head()

id  qid1  qid2                                          question1  \
0   0     1     2  What is the step by step guide to invest in sh...   
1   1     3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   
2   2     5     6  How can I increase the speed of my internet co...   
3   3     7     8  Why am I mentally very lonely? How can I solve...   
4   4     9    10  Which one dissolve in water quikly sugar, salt...   

                                           question2  is_duplicate  \
0  What is the step by step guide to invest in sh...             0   
1  What would happen if the Indian government sto...             0   
2  How can Internet speed be increased by hacking...             0   
3  Find the remainder when [math]23^{24}[/math] i...             0   
4            Which fish would survive in salt water?             0   

                                          q1_feats_m  
0  [89.86769258975983, 70.37570208311081, 45.0121...  
1  [-25.952563285827637, 31.199272722005844, 35.3...  
2  [-6.839347839355469, 53.912355184555054, 4.063...  
3  [4.993301630020142, -28.04409608244896, 39.993...  
4  [29.725815564393997, 30.60456481203437, 71.452...

In [107]:
df

id    qid1    qid2  \
0            0       1       2   
1            1       3       4   
2            2       5       6   
3            3       7       8   
4            4       9      10   
5            5      11      12   
6            6      13      14   
7            7      15      16   
8            8      17      18   
9            9      19      20   
10          10      21      22   
11          11      23      24   
12          12      25      26   
13          13      27      28   
14          14      29      30   
15          15      31      32   
16          16      33      34   
17          17      35      36   
18          18      37      38   
19          19      39      40   
20          20      41      42   
21          21      43      44   
22          22      45      46   
23          23      47      48   
24          24      49      50   
25          25      51      52   
26          26      53      54   
27          27      55      56   
28          28      57      58   
29          29      59      60   
...        ...     ...     ...   
99971    99971  165886   76371   
99972    99972  165887   95159   
99973    99973  165888  165889   
99974    99974  165890   74772   
99975    99975  165891  165892   
99976    99976  165893  165894   
99977    99977  104906  165895   
99978    99978  150224  165896   
99979    99979  165897  165898   
99980    99980  165899  165900   
99981    99981  165901  165902   
99982    99982  165903  165904   
99983    99983  165905   17577   
99984    99984  165906  165907   
99985    99985  165908  133295   
99986    99986  140081  165909   
99987    99987  165910  165911   
99988    99988   30057    9835   
99989    99989  165912  165913   
99990    99990  165914  165915   
99991    99991  165916  165917   
99992    99992  144232  165918   
99993    99993  158293  165919   
99994    99994  165920  165921   
99995    99995  165922  165923   
99996    99996  165924  165925   
99997    99997  165926  165927   
99998    99998  165928  165929   
99999    99999  165930  165931   
100000  100000  165932  165933   

                                                question1  \
0       What is the step by step guide to invest in sh...   
1       What is the story of Kohinoor (Koh-i-Noor) Dia...   
2       How can I increase the speed of my internet co...   
3       Why am I mentally very lonely? How can I solve...   
4       Which one dissolve in water quikly sugar, salt...   
5       Astrology: I am a Capricorn Sun Cap moon and c...   
6                                     Should I buy tiago?   
7                          How can I be a good geologist?   
8                         When do you use シ instead of し?   
9       Motorola (company): Can I hack my Charter Moto...   
10      Method to find separation of slits using fresn...   
11            How do I read and find my YouTube comments?   
12                   What can make Physics easy to learn?   
13            What was your first sexual experience like?   
14      What are the laws to change your status from a...   
15      What would a Trump presidency mean for current...   
16                           What does manipulation mean?   
17      Why do girls want to be friends with the guy t...   
18      Why are so many Quora users posting questions ...   
19      Which is the best digital marketing institutio...   
20                             Why do rockets look white?   
21                  What's causing someone to be jealous?   
22        What are the questions should not ask on Quora?   
23                               How much is 30 kV in HP?   
24      What does it mean that every time I look at th...   
25      What are some tips on making it through the jo...   
26                               What is web application?   
27      Does society place too much importance on sports?   
28                 What is best way to make money online?   
29                 How should I prepare for CA final law?   
...         

In [42]:
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc2), 384])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

100%|████████████████████████████████████████████████████████████████████████████████| 101/101 [00:01<00:00, 59.49it/s]


In [43]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [46]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [47]:
df3_q1

0           1           2           3          4          5    \
0    89.867693   70.375702   45.012152   96.039069 -30.653307  16.416699   
1   -25.952563   31.199273   35.315209   66.402004 -22.323479  14.264971   
2    -6.839348   53.912355    4.063398   74.270878   3.926634  15.151205   
3     4.993302  -28.044096   39.993214   51.560366  33.262052 -32.299409   
4    29.725816   30.604565   71.452822   48.382885 -22.297959  20.536037   
5     1.168779   79.307040   93.637717   82.264166  -0.789287  57.044991   
6    26.501219    3.138452   -5.626414   20.745479  10.347528 -14.271012   
7    10.131127    7.622159  -20.621897   53.921360   2.112708 -10.363263   
8    23.124455   10.926171    6.375194    6.608734  25.571843 -33.670690   
9   -34.858955   68.720824   76.893188   45.534660 -25.523559  88.492926   
10    7.347438   33.199297   -1.955804   53.311844 -43.967765  65.627578   
11  -17.213633   32.476400   39.379244   18.697383 -14.062564  66.141853   
12   24.847587   22.823173   38.704704   59.631343 -22.997522  26.321054   
13    1.255482  -17.777085  -22.358743   72.558001   6.617941 -20.017256   
14   97.446921   71.688220  109.551994  239.829353 -59.776455  83.786406   
15  -29.486470   72.864840    5.200376  112.208076 -69.515543  19.004201   
16   35.917506   15.452053  -17.544661   34.915163   9.688629  34.084496   
17   40.450115    0.435007   34.116890   65.171883 -33.063330  48.367389   
18  -19.641950   30.572932   54.929450   -3.591658   8.484625 -93.147743   
19   -9.372535    9.583969   15.150090   74.296224   6.822787 -15.727786   
20   10.199023   28.655668  -11.128129    3.086209 -12.680686  24.611893   
21  -19.887495   -3.226054    6.397642   64.214102   1.870228  -2.233511   
22    9.140100    7.686676   35.369088   42.703959  13.608345   0.888187   
23   -4.781566   26.046930   64.496153   51.957133  14.081462 -18.580094   
24   17.028434   37.566699   46.400093  101.018806  17.348282 -34.041039   
25   48.798174   48.290077   39.048552  142.559182  11.206989 -27.625079   
26   19.170014   17.282300  -13.151680   39.194928   7.496837  -0.823206   
27    6.176656   35.388137   12.855378   62.367623  -9.346206  27.423545   
28   22.708765   17.678964    9.738579   97.560940  23.099331  -6.841177   
29   -0.090399   42.095649    5.982704   32.638654 -16.665911  11.368034   
..         ...         ...         ...         ...        ...        ...   
71  -10.426638   -6.913538   -9.471499   59.622009 -15.189539   9.368341   
72   -2.718599   47.675476    5.673413   30.341381   4.082391  40.979856   
73  -17.751327   63.043444    6.244719   74.994069  -3.814920  62.212767   
74   28.378027   47.650243   -9.201591  109.478516  20.654523 -53.210166   
75   44.652671   13.374819   16.812462  161.114583  53.549479  11.003221   
76   35.725509   39.462964    0.186298   12.028820   6.939204  12.867353   
77   -2.374092   73.164539   44.135921   46.575926  14.940119   1.850778   
78   40.529553   25.746214   34.350208   72.527424  15.511139  -9.128019   
79   14.588769    7.312335    5.465239   36.649121   9.405249 -22.995039   
80  -13.619357   92.444889   93.023783   82.355475 -38.565055  -6.747609   
81  -34.587525   15.822827   17.420698  138.781719  20.955204  13.077286   
82   45.022943   96.482560   65.161052  151.115174 -42.073045  87.292236   
83   18.534271   62.425413   17.300450   -4.470637 -29.730456  14.389256   
84   29.249218   16.637615   43.791837   84.288569 -36.315253 -51.768571   
85  -16.277033   24.497298   14.464912   10.581642  25.833313 -36.315123   
86   19.786769   79.367736   44.277746   53.567819 -38.533166 -24.348396   
87   27.053868   29.315400   -1.465295   55.839620   3.178183   3.682149   
88  -28.554162   10.692697   20.377900   60.941058  32.358479 -31.125485   
89    9.072344   91.342494   33.171809   95.616093 -41.412679 -23.705791   
90   25.462121   23.863029    6.227096   69.281120  27.112040   4.999686   
91  -90.012008   29.332822  104.337979  121.751954 -52.3594

In [48]:
# dataframe of nlp features
df1.head()

id  is_duplicate   cwc_min   cwc_max   csc_min   csc_max   ctc_min  \
0   0             0  0.999980  0.833319  0.999983  0.999983  0.916659   
1   1             0  0.799984  0.399996  0.749981  0.599988  0.699993   
2   2             0  0.399992  0.333328  0.399992  0.249997  0.399996   
3   3             0  0.000000  0.000000  0.000000  0.000000  0.000000   
4   4             0  0.399992  0.199998  0.999950  0.666644  0.571420   

    ctc_max  last_word_eq  first_word_eq  abs_len_diff  mean_len  \
0  0.785709           0.0            1.0           2.0      13.0   
1  0.466664           0.0            1.0           5.0      12.5   
2  0.285712           0.0            1.0           4.0      12.0   
3  0.000000           0.0            0.0           2.0      12.0   
4  0.307690           0.0            1.0           6.0      10.0   

   token_set_ratio  token_sort_ratio  fuzz_ratio  fuzz_partial_ratio  \
0              100                93          93                 100   
1               86                63          66                  75   
2               66                66          54                  54   
3               36                36          35                  40   
4               67                47          46                  56   

   longest_substr_ratio  
0              0.982759  
1              0.596154  
2              0.166667  
3              0.039216  
4              0.175000

In [49]:
# data before preprocessing 
df2.head()

id  freq_qid1  freq_qid2  q1len  q2len  q1_n_words  q2_n_words  \
0   0          1          1     66     57          14          12   
1   1          4          1     51     88           8          13   
2   2          1          1     73     59          14          10   
3   3          1          1     50     65          11           9   
4   4          3          1     76     39          13           7   

   word_Common  word_Total  word_share  freq_q1+q2  freq_q1-q2  
0         10.0        23.0    0.434783           2           0  
1          4.0        20.0    0.200000           5           3  
2          4.0        24.0    0.166667           2           0  
3          0.0        19.0    0.000000           2           0  
4          2.0        20.0    0.100000           4           2

In [50]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

0          1          2          3          4          5    \
0  89.867693  70.375702  45.012152  96.039069 -30.653307  16.416699   
1 -25.952563  31.199273  35.315209  66.402004 -22.323479  14.264971   
2  -6.839348  53.912355   4.063398  74.270878   3.926634  15.151205   
3   4.993302 -28.044096  39.993214  51.560366  33.262052 -32.299409   
4  29.725816  30.604565  71.452822  48.382885 -22.297959  20.536037   

          6          7          8          9    ...        374        375  \
0 -123.361826 -68.260223  85.066650  29.747428  ...   8.351350  27.336714   
1  -20.489592 -44.117097  36.284951  -1.185126  ...  -9.313895  -4.944982   
2 -106.160012 -67.004788  26.851934  33.661832  ...   2.893046  10.220280   
3  -21.068064   7.457745  -2.452946 -21.497446  ...  10.652281   0.016257   
4 -179.450682 -64.894803  53.710510  50.643988  ...   6.058635  23.819845   

        376        377        378       379        380       381       382  \
0  6.181333 -11.127490  13.250205  6.628089 -19.777541  4.143553 -2.198851   
1  8.472605  -0.820464  -8.585880 -2.045869  -6.408547 -0.560994 -6.804029   
2 -3.133157  -3.110367   3.093527  3.314425 -14.921359  7.886443  9.744542   
3 -0.811031  -6.263776  14.141930  9.989349  -6.887892  3.909686  8.648553   
4  6.553738 -12.311948  22.809184  5.550088 -20.109400  4.709517  4.630830   

        383  
0 -0.214953  
1 -2.765754  
2 -6.512999  
3  4.257667  
4  2.415642  

[5 rows x 384 columns]

In [51]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

0          1          2          3          4          5    \
0  91.087929  67.169421  21.091323  77.112044 -22.115122  22.450940   
1 -51.720582  47.688753  42.838216  56.544479 -38.038384  60.271469   
2   9.939538   7.660260  -1.570720  53.088465 -17.804865  55.187640   
3  31.150765  24.686227 -15.263172  54.650256  10.007230 -13.452440   
4  17.296943  46.998401   0.508177  35.755096 -35.367800  46.271158   

          6          7          8          9    ...        374        375  \
0 -101.674162 -61.779865  79.051499  40.339972  ...  11.399936  21.225864   
1  -91.386137 -68.484431  27.785514  14.757023  ...   5.078152  -2.731408   
2  -83.190190 -78.806606  38.630674  30.456415  ...   6.654874  -1.039904   
3  -41.234129 -28.708703  32.136721 -18.060924  ...   3.511406   1.114447   
4  -80.402842 -17.151527  46.547046  42.535995  ...  -2.825094   7.873711   

         376        377        378        379        380       381       382  \
0   4.966831 -10.459152  11.432489   8.144732 -16.050024  2.242944  0.778122   
1  25.308943  -2.251558  -3.434343  -2.683669   3.221457 -0.471526  7.528792   
2   7.018560 -10.581887  10.414846  11.424499  -0.689837  9.709516  9.345433   
3   8.269716  -4.857381   4.103357   5.216257   2.758758 -2.219298  3.749893   
4   5.572179 -10.629673   5.702092   1.198984  -3.890753  7.755348  8.117149   

        383  
0  1.382590  
1  1.496061  
2  5.655827  
3  5.329140  
4  1.853889  

[5 rows x 384 columns]

In [52]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
Number of features in question1 w2v  dataframe : 384
Number of features in question2 w2v  dataframe : 384
Number of features in final dataframe  : 797


In [53]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('final_features.csv')

In [1]:
import gensim
import gensim.downloader as api

In [5]:
word2vec_model300 = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [14]:
len(word2vec_model300[''])

300